In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
import re
import time
import nltk
from tqdm import tqdm
import lightgbm as lgb
tqdm.pandas()
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

In [5]:
ruddit_df = pd.read_csv("../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv")
toxic_classification_test = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv")
toxic_classification_test_labels = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
toxic_classification_train = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv")

data_path = "/kaggle/input/jigsaw-toxic-severity-rating/"
validation_data = pd.read_csv(data_path+"validation_data.csv")
final_data = pd.read_csv(data_path+"comments_to_score.csv")

## Prepping the ruddit dataset

In [6]:
ruddit_df.head()

In [7]:
ruddit_df.shape

### Looking at distribution of offensive scores

In [8]:
fig, ax = plt.subplots()
ruddit_df.offensiveness_score.plot(ax=ax, kind='hist', color='green', bins=50, alpha=0.7)
plt.show()

### Writing a custom scoring function to convert offensiveness scores to toxicity scores

In [9]:
def custom_rank(score):
    if score<=0:
        return 0
    else:
        if score<0.2:
            return 1
        elif score>=0.2 and score<0.4:
            return 2
        elif score>=0.4 and score<0.7:
            return 3
        elif score>=0.7 and score<=1:
            return 4

ruddit_df['toxicity'] = ruddit_df['offensiveness_score'].progress_apply(custom_rank)

In [10]:
ruddit_df.columns.values

In [11]:
del(ruddit_df['post_id'])
del(ruddit_df['comment_id'])
del(ruddit_df['url'])
del(ruddit_df['offensiveness_score'])

In [12]:
ruddit_df.toxicity.value_counts()

In [13]:
ruddit_df = ruddit_df.rename(columns={'txt':'comment_text'})

In [14]:
df_test = pd.merge(toxic_classification_test, toxic_classification_test_labels, how='left', on='id')
df = pd.concat([toxic_classification_train, df_test])

df = df[(df['toxic']!=-1) & (df['severe_toxic']!=-1) & (df['obscene']!=-1) & (df['threat']!=-1) & (df['insult']!=-1) & (df['identity_hate']!=-1)]
df['toxicity'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)
df['toxicity'] = df['toxicity'].apply(lambda x: 4 if x in [4,5,6] else x)

In [15]:
del(df['id'])
del(df['toxic'])
del(df['identity_hate'])
del(df['severe_toxic'])
del(df['obscene'])
del(df['threat'])
del(df['insult'])

In [16]:
df = pd.concat([df, ruddit_df])

In [17]:
df.toxicity.value_counts()

## Cleaning the comments

In [18]:
def clean(data, col):
    
    data[col] = data[col].str.replace('https?://\S+|www\.\S+', ' social medium ')      
        
    data[col] = data[col].str.lower()
    data[col] = data[col].str.replace("4", "a") 
    data[col] = data[col].str.replace("2", "l")
    data[col] = data[col].str.replace("5", "s") 
    data[col] = data[col].str.replace("1", "i") 
    data[col] = data[col].str.replace("!", "i") 
    data[col] = data[col].str.replace("|", "i") 
    data[col] = data[col].str.replace("0", "o") 
    data[col] = data[col].str.replace("l3", "b") 
    data[col] = data[col].str.replace("7", "t") 
    data[col] = data[col].str.replace("7", "+") 
    data[col] = data[col].str.replace("8", "ate") 
    data[col] = data[col].str.replace("3", "e") 
    data[col] = data[col].str.replace("9", "g")
    data[col] = data[col].str.replace("6", "g")
    data[col] = data[col].str.replace("@", "a")
    data[col] = data[col].str.replace("$", "s")
    data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")    
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace("centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", " sick ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
    data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
    data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ")
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
    data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ")
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you')
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " is ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace('\s+', ' ')  # will remove more than one whitespace character
#     text = re.sub(r'\b([^\W\d_]+)(\s+\1)+\b', r'\1', re.sub(r'\W+', ' ', text).strip(), flags=re.I)  # remove repeating words coming immediately one after another
    data[col] = data[col].str.replace(r'(.)\1+', r'\1\1') # 2 or more characters are replaced by 2 characters
#     text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
    data[col] = data[col].str.replace("[:|♣|'|§|♠|*|/|?|=|%|&|-|#|•|~|^|>|<|►|_]", '')
    
    
    data[col] = data[col].str.replace(r"what's", "what is ")    
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace(r"\'s", " ")

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    return data

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+')  # Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml')  # Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) # Remove special Charecters
    text = re.sub(' +', ' ', text) # Remove Extra Spaces
    text = text.strip().lower() # remove spaces at the beginning and at the end of string and make string lower
    
    # lemmatization
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])
    # del stopwords
    text = ' '.join([word for word in text.split(' ') if word not in stop])

    return text

In [19]:
sampled_df = df.groupby('toxicity', group_keys=False).apply(lambda x: x.sample(min(len(x), 9000)))

In [20]:
sampled_df = clean(sampled_df, 'comment_text')
sampled_df['clean_text'] = sampled_df['comment_text'].progress_apply(text_cleaning)

## Fitting a TF-IDF + lightGBM model

In [24]:
vectorizer = TfidfVectorizer(min_df=3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5))
comments_tr = vectorizer.fit_transform(sampled_df.clean_text)
comments_tr

### Doing SVD to bring down the number of features

In [ ]:

# comments_tr = StandardScaler(with_mean=False).fit_transform(comments_tr)

# pca = PCA(n_components=10000)
# principalComponents = pca.fit_transform(comments_tr)

# svd = TruncatedSVD(n_components=10000, random_state=42)

In [ ]:
# X_train, X_dev, y_train, y_dev = train_test_split(comments_tr.toarray(), sampled_df.toxicity, test_size=0.25, random_state=42)

In [28]:
start = time.time()
model = lgb.LGBMClassifier(learning_rate=0.05,random_state=42)
model.fit(comments_tr,sampled_df.toxicity,
          verbose=20,eval_metric='logloss')

end = time.time()

print("Time taken:", end-start)

In [29]:
def get_rank_score(vecs):
    labels = model.predict(vecs)
    prob_scores = model.predict_proba(vecs)
    rank_scores = labels + np.max(prob_scores, axis=1)
    return rank_scores

less_toxic_vecs = vectorizer.transform(validation_data.less_toxic)
more_toxic_vecs = vectorizer.transform(validation_data.more_toxic)

less_toxic_score = get_rank_score(less_toxic_vecs)
more_toxic_score = get_rank_score(more_toxic_vecs)

In [31]:
np.sum(less_toxic_score<more_toxic_score)*100/validation_data.shape[0]